## Gathering Data

In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import time

In [2]:
# download tsv file from Udacity's server using requests
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open (url.split('/')[-1], mode='wb') as file:
    file.write(response.content)

In [2]:
image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [3]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
consumer_key = 'MY CONSUMER KEY'
consumer_secret = 'MY CONSUMER SECRET'
access_token = 'MY ACCESS TOKEN'
access_secret = 'MY ACCESS SECRET'

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth,
                 parser=tweepy.parsers.JSONParser(),
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

In [15]:
error_list = list()
query_success_count = 0
for tweet_id in twitter_archive.tweet_id:
    try:
        # get the start time of query
        start_time = time.time()
        # query each id using get_status
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        # open the tweet_json.txt file in append mode
        with open('tweet_json.txt', 'a') as file:
            # write each tweet's JSON data in a new line
            file.write(json.dumps(tweet) + '\n')
        # count the number of success in querying the status and recording its data
        query_success_count += 1
        # get the end time of query
        end_time = time.time()
        print('Query for id {} successful. Time taken (s): {}'.format(tweet_id, end_time-start_time))
    except Exception as e:
        # record exceptions in a list of dictionaries
        error_list.append({'tweet_id': int(tweet_id), 'error': e})

Query for id 892420643555336193 successful. Time taken (s): 1.1852355003356934
Query for id 892177421306343426 successful. Time taken (s): 1.2132792472839355
Query for id 891815181378084864 successful. Time taken (s): 1.1902949810028076
Query for id 891689557279858688 successful. Time taken (s): 1.1611762046813965
Query for id 891327558926688256 successful. Time taken (s): 1.1504487991333008
Query for id 891087950875897856 successful. Time taken (s): 1.1227457523345947
Query for id 890971913173991426 successful. Time taken (s): 1.2379348278045654
Query for id 890729181411237888 successful. Time taken (s): 1.1992018222808838
Query for id 890609185150312448 successful. Time taken (s): 1.1760127544403076
Query for id 890240255349198849 successful. Time taken (s): 1.151318073272705
Query for id 890006608113172480 successful. Time taken (s): 1.1513917446136475
Query for id 889880896479866881 successful. Time taken (s): 1.2107594013214111
Query for id 889665388333682689 successful. Time take

Query for id 871762521631449091 successful. Time taken (s): 1.145768642425537
Query for id 871515927908634625 successful. Time taken (s): 1.2127444744110107
Query for id 871166179821445120 successful. Time taken (s): 1.1907405853271484
Query for id 871102520638267392 successful. Time taken (s): 1.2331526279449463
Query for id 871032628920680449 successful. Time taken (s): 1.1787889003753662
Query for id 870804317367881728 successful. Time taken (s): 1.1947920322418213
Query for id 870726314365509632 successful. Time taken (s): 1.1917991638183594
Query for id 870656317836468226 successful. Time taken (s): 1.1348450183868408
Query for id 870374049280663552 successful. Time taken (s): 1.2318243980407715
Query for id 870308999962521604 successful. Time taken (s): 1.224966049194336
Query for id 870063196459192321 successful. Time taken (s): 1.14835524559021
Query for id 869772420881756160 successful. Time taken (s): 1.1925685405731201
Query for id 869702957897576449 successful. Time taken (

Query for id 851224888060895234 successful. Time taken (s): 1.1670148372650146
Query for id 850753642995093505 successful. Time taken (s): 1.1688942909240723
Query for id 850380195714523136 successful. Time taken (s): 1.1599671840667725
Query for id 850333567704068097 successful. Time taken (s): 1.1520588397979736
Query for id 850145622816686080 successful. Time taken (s): 1.1975524425506592
Query for id 850019790995546112 successful. Time taken (s): 1.1616687774658203
Query for id 849776966551130114 successful. Time taken (s): 1.354525089263916
Query for id 849668094696017920 successful. Time taken (s): 1.1558332443237305
Query for id 849412302885593088 successful. Time taken (s): 1.166093111038208
Query for id 849336543269576704 successful. Time taken (s): 1.154222011566162
Query for id 849051919805034497 successful. Time taken (s): 1.228428602218628
Query for id 848690551926992896 successful. Time taken (s): 1.1389122009277344
Query for id 848324959059550208 successful. Time taken (

Query for id 834089966724603904 successful. Time taken (s): 1.1559500694274902
Query for id 834086379323871233 successful. Time taken (s): 1.3236644268035889
Query for id 833863086058651648 successful. Time taken (s): 1.1702980995178223
Query for id 833826103416520705 successful. Time taken (s): 1.145967721939087
Query for id 833732339549220864 successful. Time taken (s): 1.1790876388549805
Query for id 833722901757046785 successful. Time taken (s): 1.1399872303009033
Query for id 833479644947025920 successful. Time taken (s): 1.104651927947998
Query for id 833124694597443584 successful. Time taken (s): 1.1402578353881836
Query for id 832998151111966721 successful. Time taken (s): 1.154421329498291
Query for id 832769181346996225 successful. Time taken (s): 1.150604486465454
Query for id 832757312314028032 successful. Time taken (s): 1.1569936275482178
Query for id 832682457690300417 successful. Time taken (s): 1.2143440246582031
Query for id 832645525019123713 successful. Time taken (

Query for id 821149554670182400 successful. Time taken (s): 1.211503028869629
Query for id 821107785811234820 successful. Time taken (s): 1.1782851219177246
Query for id 821044531881721856 successful. Time taken (s): 1.2138144969940186
Query for id 820837357901512704 successful. Time taken (s): 1.153977870941162
Query for id 820749716845686786 successful. Time taken (s): 1.1434612274169922
Query for id 820690176645140481 successful. Time taken (s): 1.1791794300079346
Query for id 820494788566847489 successful. Time taken (s): 1.2334086894989014
Query for id 820446719150292993 successful. Time taken (s): 1.1727635860443115
Query for id 820314633777061888 successful. Time taken (s): 1.1849534511566162
Query for id 820078625395449857 successful. Time taken (s): 1.1640617847442627
Query for id 820013781606658049 successful. Time taken (s): 1.175201416015625
Query for id 819952236453363712 successful. Time taken (s): 1.2714166641235352
Query for id 819924195358416896 successful. Time taken 

...

Query for id 674739953134403584 successful. Time taken (s): 1.1387686729431152
Query for id 674737130913071104 successful. Time taken (s): 1.252838134765625
Query for id 674690135443775488 successful. Time taken (s): 1.237830400466919
Query for id 674670581682434048 successful. Time taken (s): 1.34590744972229
Query for id 674664755118911488 successful. Time taken (s): 1.1597652435302734
Query for id 674646392044941312 successful. Time taken (s): 1.101738691329956
Query for id 674644256330530816 successful. Time taken (s): 1.2078096866607666
Query for id 674638615994089473 successful. Time taken (s): 1.1667871475219727
Query for id 674632714662858753 successful. Time taken (s): 1.1577703952789307
Query for id 674606911342424069 successful. Time taken (s): 1.1877944469451904
Query for id 674468880899788800 successful. Time taken (s): 1.2718498706817627
Query for id 674447403907457024 successful. Time taken (s): 1.3549096584320068
Query for id 674436901579923456 successful. Time taken (s

Query for id 672482722825261057 successful. Time taken (s): 1.1827878952026367
Query for id 672481316919734272 successful. Time taken (s): 1.1978018283843994
Query for id 672475084225949696 successful. Time taken (s): 1.1607770919799805
Query for id 672466075045466113 successful. Time taken (s): 1.1697826385498047
Query for id 672272411274932228 successful. Time taken (s): 1.1247541904449463
Query for id 672267570918129665 successful. Time taken (s): 1.1797866821289062
Query for id 672264251789176834 successful. Time taken (s): 1.1627779006958008
Query for id 672256522047614977 successful. Time taken (s): 1.1867921352386475
Query for id 672254177670729728 successful. Time taken (s): 1.163780927658081
Query for id 672248013293752320 successful. Time taken (s): 1.1447653770446777
Query for id 672245253877968896 successful. Time taken (s): 1.231828212738037
Query for id 672239279297454080 successful. Time taken (s): 1.2088098526000977
Query for id 672231046314901505 successful. Time taken

Query for id 670468609693655041 successful. Time taken (s): 1.1937978267669678
Query for id 670465786746662913 successful. Time taken (s): 1.1813206672668457
Query for id 670452855871037440 successful. Time taken (s): 1.134617805480957
Query for id 670449342516494336 successful. Time taken (s): 1.2120399475097656
Query for id 670444955656130560 successful. Time taken (s): 1.2240335941314697
Query for id 670442337873600512 successful. Time taken (s): 1.1840319633483887
Query for id 670435821946826752 successful. Time taken (s): 1.1840369701385498
Query for id 670434127938719744 successful. Time taken (s): 1.1560306549072266
Query for id 670433248821026816 successful. Time taken (s): 1.3200385570526123
Query for id 670428280563085312 successful. Time taken (s): 1.2680315971374512
Query for id 670427002554466305 successful. Time taken (s): 1.220031976699829
Query for id 670421925039075328 successful. Time taken (s): 1.1880381107330322
Query for id 670420569653809152 successful. Time taken

Query for id 668620235289837568 successful. Time taken (s): 1.2160329818725586
Query for id 668614819948453888 successful. Time taken (s): 1.1840343475341797
Query for id 668587383441514497 successful. Time taken (s): 1.188037633895874
Query for id 668567822092664832 successful. Time taken (s): 1.2040274143218994
Query for id 668544745690562560 successful. Time taken (s): 1.1840391159057617
Query for id 668542336805281792 successful. Time taken (s): 1.4000446796417236
Query for id 668537837512433665 successful. Time taken (s): 1.1320304870605469
Query for id 668528771708952576 successful. Time taken (s): 1.1840341091156006
Query for id 668507509523615744 successful. Time taken (s): 1.1480324268341064
Query for id 668496999348633600 successful. Time taken (s): 1.1680376529693604
Query for id 668484198282485761 successful. Time taken (s): 1.2120387554168701
Query for id 668480044826800133 successful. Time taken (s): 1.1840345859527588
Query for id 668466899341221888 successful. Time take

Query for id 666691418707132416 successful. Time taken (s): 1.1995396614074707
Query for id 666649482315059201 successful. Time taken (s): 2.2280585765838623
Query for id 666644823164719104 successful. Time taken (s): 1.1720342636108398
Query for id 666454714377183233 successful. Time taken (s): 1.2200393676757812
Query for id 666447344410484738 successful. Time taken (s): 1.1680283546447754
Query for id 666437273139982337 successful. Time taken (s): 1.1760375499725342
Query for id 666435652385423360 successful. Time taken (s): 1.1720325946807861
Query for id 666430724426358785 successful. Time taken (s): 1.1640386581420898
Query for id 666428276349472768 successful. Time taken (s): 1.160033941268921
Query for id 666421158376562688 successful. Time taken (s): 1.1600377559661865
Query for id 666418789513326592 successful. Time taken (s): 1.1680269241333008
Query for id 666411507551481857 successful. Time taken (s): 1.1560389995574951
Query for id 666407126856765440 successful. Time take

In [16]:
query_success_count

2345

In [17]:
# read the .txt file, line by line, to create a pandas DataFrame that you will soon assess and clean
df_list = list()
with open('tweet_json.txt') as json_file:
    for i in range(query_success_count):
        data = json_file.readline()
        json_data = json.loads(data)
        tweet_id = json_data['id']
        fav_count = json_data['favorite_count']
        ret_count = json_data['retweet_count']
        df_list.append({'tweet_id': int(tweet_id),
                        'favorite_count': int(fav_count),
                        'retweet_count': int(ret_count)})

df = pd.DataFrame(df_list, columns = ['tweet_id', 'favorite_count', 'retweet_count'])
df.head()

,tweet_id,favorite_count,retweet_count
0,892420643555336193,38835,8598
1,892177421306343426,33263,6318
2,891815181378084864,25061,4192
3,891689557279858688,42211,8713
4,891327558926688256,40360,9485


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 3 columns):
tweet_id          2345 non-null int64
favorite_count    2345 non-null int64
retweet_count     2345 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


In [19]:
# save the dataframe to a csv file
df.to_csv('fav_ret_count.csv', index=False)

In [20]:
# convert the list of exceptions into a dataframe
error_df = pd.DataFrame(error_list, columns = ['tweet_id', 'error'])
error_df

,tweet_id,error
0,888202515573088257,"[{'code': 144, 'message': 'No status found wit..."
1,873697596434513921,"[{'code': 144, 'message': 'No status found wit..."
2,869988702071779329,"[{'code': 144, 'message': 'No status found wit..."
3,866816280283807744,"[{'code': 144, 'message': 'No status found wit..."
4,861769973181624320,"[{'code': 144, 'message': 'No status found wit..."
5,845459076796616705,"[{'code': 144, 'message': 'No status found wit..."
6,842892208864923648,"[{'code': 144, 'message': 'No status found wit..."
7,837012587749474308,"[{'code': 144, 'message': 'No status found wit..."
8,827228250799742977,"[{'code': 144, 'message': 'No status found wit..."
9,802247111496568832,"[{'code': 144, 'message': 'No status found wit..."


In [27]:
# save the exceptions dataframe to a csv file
error_df.to_csv('query_errors.csv', index=False)

## Assessing Data

- Missing Data
- Tidiness
- Quality

#### Key points

Key points to keep in mind when data wrangling for this project:

- You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
- Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 quality issues in this dataset.
- Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html).
- The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.
- You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.

Detect and document at least eight (8) quality issues and two (2) tidiness issues

In [4]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
fav_retweet_counts = pd.read_csv('fav_ret_count.csv')

In [7]:
twitter_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [5]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [6]:
twitter_archive[twitter_archive.in_reply_to_status_id.notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ladies and gentlemen... I found Pipsy. He may ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547...,14,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610...,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None


In [8]:
twitter_archive[twitter_archive.retweeted_status_id.notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None


In [12]:
twitter_archive.source.unique()

array([ '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'], dtype=object)

### Quality
#### *twitter_archive* table
- in_reply_to_status_id, in_reply_to_user_id are in floats format instead of integer
- retweeted_status_id, retweeted_status_user_id are in floats format instead of integer
- timestamp and retweeted_status_timestamp are in strings format instead of datetime
- source values contain html tags

## References

- [Convert Tweepy Status Object into JSON](https://stackoverflow.com/questions/27900451/convert-tweepy-status-object-into-json)
- The structure of the Status object of Tweepy
 https://gist.github.com/dev-techmoe/ef676cdd03ac47ac503e856282077bf2
- Twitter API - get tweets with specific id
 https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id
- Tweepy documentation http://tweepy.readthedocs.io/en/v3.5.0/index.html
- [Reading and Writing JSON to a File in Python](http://stackabuse.com/reading-and-writing-json-to-a-file-in-python/)